In [1]:
import pandas as pd
import duckdb
%load_ext sql
%config SqlMagic.displaylimit = None
con = duckdb.connect("../database/social-buzz.db")
%sql con --alias duckdb

displaylimit: Value None will be treated as 0 (no limit)

In [2]:
%%sql
-- export data from source into database
CREATE TABLE content AS
SELECT * FROM read_csv("../raw-data/Content.csv", header = true);

CREATE TABLE reactions AS
SELECT * FROM read_csv("../raw-data/Reactions.csv", header = true);

CREATE TABLE reaction_type AS
SELECT * FROM read_csv("../raw-data/ReactionTypes.csv", header = true);

Running query in 'duckdb'

Count
16


Start data cleaning for each table

In [3]:
%%sql
describe content;

Running query in 'duckdb'

column_name,column_type,null,key,default,extra
column0,BIGINT,YES,None,None,None
Content ID,VARCHAR,YES,None,None,None
User ID,VARCHAR,YES,None,None,None
Type,VARCHAR,YES,None,None,None
Category,VARCHAR,YES,None,None,None
URL,VARCHAR,YES,None,None,None


In [4]:
%%sql
-- standardise fields name
ALTER TABLE content
RENAME COLUMN "Content ID" TO content_id;

ALTER TABLE content
RENAME COLUMN "User ID" to user_id;

ALTER TABLE content
RENAME COLUMN Type to content_type;

ALTER TABLE content
RENAME COLUMN Category to category;

ALTER TABLE content
RENAME COLUMN URL to url;

Running query in 'duckdb'

Success


In [5]:
%%sql
describe content

Running query in 'duckdb'

column_name,column_type,null,key,default,extra
column0,BIGINT,YES,None,None,None
content_id,VARCHAR,YES,None,None,None
user_id,VARCHAR,YES,None,None,None
content_type,VARCHAR,YES,None,None,None
category,VARCHAR,YES,None,None,None
url,VARCHAR,YES,None,None,None


In [6]:
%%sql
-- check for null values
SELECT *
FROM content
WHERE column0 ISNULL
    OR content_id ISNULL
    OR user_id ISNULL
    OR content_type ISNULL
    OR category ISNULL
    OR url ISNULL;

Running query in 'duckdb'

column0,content_id,user_id,content_type,category,url
5,cf1e8c1a-23eb-4426-9f58-002fb1b53e91,4607d7b0-3313-49b8-9f73-5b8227fc5b67,GIF,cooking,None
10,5118e9c5-1377-4cc5-a486-65b35b7b7b76,ef56931b-540b-4166-9090-73ae48ac8d97,GIF,science,None
15,7ffd0a82-4a0a-4527-a4d6-e251b756bac7,425b7021-0409-4358-af39-2bff14197d0a,GIF,fitness,None
20,baa2db00-81ba-4c34-b17d-dd950bfa133b,2f112802-2501-45b5-a9d9-470cda6ef23e,GIF,soccer,None
25,a4067fb5-3ba8-4f97-87c2-ddad5e4029c2,e2377b94-f06a-49ed-a266-377c4b07e980,GIF,food,None
30,a2ff9bf2-1fa6-4001-9566-f597f8e754ef,f0ef81ec-85b1-4475-ae60-aa7d373088aa,video,food,None
35,07f88a73-aef2-45fd-8b5d-418e448b853d,18c56602-937e-4ff3-bc6c-43f0b14212ca,GIF,Animals,None
40,ab4c4756-1c50-4136-bad9-3216e01ffac2,bfe23c39-f814-4282-b0f7-506cd8295b8a,audio,fitness,None
45,ea1afba2-c0a8-4d0f-ac64-91fd7a126bbc,49da0743-cb4d-4c84-b6a2-537bbe7256d1,GIF,healthy eating,None
50,2d949603-6676-4402-900b-2c2c78315ea0,4baf648a-0ad5-412d-8250-5a0ff9289fbf,audio,"""culture""",None


In [21]:
%%sql
-- to check duplicate rows
SELECT
    content_id,
    user_id,
    COUNT(*) AS count
FROM content
WHERE url IS NOT NULL
GROUP BY content_id, user_id
HAVING COUNT(*) > 1;

Running query in 'duckdb'

content_id,user_id,count


In [31]:
%%sql
SELECT DISTINCT content_type
FROM content
ORDER BY content_type;

Running query in 'duckdb'

content_type
GIF
audio
photo
video


In [32]:
%%sql
SELECT DISTINCT category
FROM content
ORDER BY category;

Running query in 'duckdb'

category
"""animals"""
"""cooking"""
"""culture"""
"""dogs"""
"""food"""
"""public speaking"""
"""science"""
"""soccer"""
"""studying"""
"""technology"""


In [35]:
%%sql
-- select columns needed (content_id, content_type & category)
SELECT
    content_id,
    content_type,
    REPLACE(LOWER(category), '"', '') AS category
FROM content
LIMIT 20;

    


Running query in 'duckdb'

content_id,content_type,category
97522e57-d9ab-4bd6-97bf-c24d952602d2,photo,studying
9f737e0a-3cdd-4d29-9d24-753f4e3be810,photo,healthy eating
230c4e4d-70c3-461d-b42c-ec09396efb3f,photo,healthy eating
356fff80-da4d-4785-9f43-bc1261031dc6,photo,technology
01ab84dd-6364-4236-abbb-3f237db77180,video,food
cf1e8c1a-23eb-4426-9f58-002fb1b53e91,GIF,cooking
3f8590c7-6ab2-4973-805a-90cdec355f05,video,dogs
e5490118-90d5-4572-ab1c-1fbc87b8d9ca,video,technology
0bedca96-fb76-4287-a83c-17330ed39cce,photo,soccer
b18cb63f-4c8e-44ee-a47f-541e95191d11,photo,public speaking


In [36]:
%%sql
--export to csv
COPY (
SELECT
    content_id,
    content_type,
    REPLACE(LOWER(category), '"', '') AS category
FROM content
)
TO "../data/clean/content-clean.csv" (HEADER, DELIMITER ',');

Running query in 'duckdb'

Count
1000


In [37]:
%%sql
describe reactions

Running query in 'duckdb'

column_name,column_type,null,key,default,extra
column0,BIGINT,YES,None,None,None
Content ID,VARCHAR,YES,None,None,None
User ID,VARCHAR,YES,None,None,None
Type,VARCHAR,YES,None,None,None
Datetime,TIMESTAMP,YES,None,None,None


In [45]:
%%sql
-- to check duplicate rows
SELECT
    "Content ID" AS content_id,
    "User ID" AS user_id,
    Datetime AS datetime,
    COUNT(*) AS count
FROM reactions
GROUP BY content_id, user_id, datetime
HAVING COUNT(*) > 1;

Running query in 'duckdb'

content_id,user_id,datetime,count


In [52]:
%%sql
SELECT DISTINCT Type
FROM reactions
ORDER BY Type;

Running query in 'duckdb'

Type
adore
cherish
disgust
dislike
hate
heart
indifferent
interested
intrigued
like


In [53]:
%%sql
-- select columns needed (content_id, reaction_type & datetime)
SELECT
    "Content ID" AS content_id,
    LOWER(Type) AS reaction_type,
    Datetime AS datetime
FROM reactions
WHERE content_id IS NOT NULL
    AND reaction_type IS NOT NULL
    AND datetime IS NOT NULL
LIMIT 20;

Running query in 'duckdb'

content_id,reaction_type,datetime
97522e57-d9ab-4bd6-97bf-c24d952602d2,disgust,2020-11-07 09:43:50
97522e57-d9ab-4bd6-97bf-c24d952602d2,dislike,2021-06-17 12:22:51
97522e57-d9ab-4bd6-97bf-c24d952602d2,scared,2021-04-18 05:13:58
97522e57-d9ab-4bd6-97bf-c24d952602d2,disgust,2021-01-06 19:13:01
97522e57-d9ab-4bd6-97bf-c24d952602d2,interested,2020-08-23 12:25:58
97522e57-d9ab-4bd6-97bf-c24d952602d2,peeking,2020-12-07 06:27:54
97522e57-d9ab-4bd6-97bf-c24d952602d2,cherish,2021-04-11 17:35:49
97522e57-d9ab-4bd6-97bf-c24d952602d2,hate,2021-01-27 08:32:09
97522e57-d9ab-4bd6-97bf-c24d952602d2,peeking,2021-04-01 22:54:23
97522e57-d9ab-4bd6-97bf-c24d952602d2,love,2020-08-04 05:05:02


In [54]:
%%sql
--export to csv
COPY (
SELECT
    "Content ID" AS content_id,
    LOWER(Type) AS reaction_type,
    Datetime AS datetime
FROM reactions
WHERE content_id IS NOT NULL
    AND reaction_type IS NOT NULL
    AND datetime IS NOT NULL
)
TO "../data/clean/reactions-clean.csv" (HEADER, DELIMITER ',');

Running query in 'duckdb'

Count
24573


In [55]:
%%sql
describe reaction_type

Running query in 'duckdb'

column_name,column_type,null,key,default,extra
column0,BIGINT,YES,None,None,None
Type,VARCHAR,YES,None,None,None
Sentiment,VARCHAR,YES,None,None,None
Score,BIGINT,YES,None,None,None


In [58]:
%%sql
SELECT *
FROM reaction_type
ORDER BY Score DESC;

Running query in 'duckdb'

column0,Type,Sentiment,Score
7,super love,positive,75
9,adore,positive,72
1,want,positive,70
8,cherish,positive,70
6,love,positive,65
0,heart,positive,60
10,like,positive,50
12,intrigued,positive,45
13,peeking,neutral,35
4,interested,positive,30


In [59]:
%%sql
-- export to csv
COPY (
SELECT
    Type AS reaction_type,
    Sentiment AS sentiment,
    Score AS score
FROM reaction_type
)
TO "../data/clean/reaction-type-clean.csv" (HEADER, DELIMITER ',');

Running query in 'duckdb'

Count
16


In [67]:
%%sql
SELECT
    r."Content ID" AS content_id,
    LOWER(r.Type) AS reaction_type,
    r.Datetime AS datetime,
    c.content_type,
    REPLACE(LOWER(c.category), '"', '') AS category
FROM reactions r
INNER JOIN content c ON r."Content ID" = c.content_id
WHERE r."Content ID" IS NOT NULL
    AND r.Type IS NOT NULL
    AND r.Datetime IS NOT NULL
LIMIT 20;

Running query in 'duckdb'

content_id,reaction_type,datetime,content_type,category
97522e57-d9ab-4bd6-97bf-c24d952602d2,disgust,2020-11-07 09:43:50,photo,studying
97522e57-d9ab-4bd6-97bf-c24d952602d2,dislike,2021-06-17 12:22:51,photo,studying
97522e57-d9ab-4bd6-97bf-c24d952602d2,scared,2021-04-18 05:13:58,photo,studying
97522e57-d9ab-4bd6-97bf-c24d952602d2,disgust,2021-01-06 19:13:01,photo,studying
97522e57-d9ab-4bd6-97bf-c24d952602d2,interested,2020-08-23 12:25:58,photo,studying
97522e57-d9ab-4bd6-97bf-c24d952602d2,peeking,2020-12-07 06:27:54,photo,studying
97522e57-d9ab-4bd6-97bf-c24d952602d2,cherish,2021-04-11 17:35:49,photo,studying
97522e57-d9ab-4bd6-97bf-c24d952602d2,hate,2021-01-27 08:32:09,photo,studying
97522e57-d9ab-4bd6-97bf-c24d952602d2,peeking,2021-04-01 22:54:23,photo,studying
97522e57-d9ab-4bd6-97bf-c24d952602d2,love,2020-08-04 05:05:02,photo,studying


In [80]:
%%sql
-- to get top 5 category
SELECT
    REPLACE(LOWER(c.category), '"', '') AS category,
    SUM(rt.Score) AS total_score
FROM
    reactions r
    INNER JOIN content c ON r."Content ID" = c.content_id
    INNER JOIN reaction_type rt ON r.Type = rt.Type
WHERE
    r."Content ID" IS NOT NULL
    AND r.Type IS NOT NULL
    AND r.Datetime IS NOT NULL
GROUP BY category
ORDER BY total_score DESC
LIMIT 5;

Running query in 'duckdb'

category,total_score
animals,69548
healthy eating,69067
technology,67472
science,66043
cooking,63982


In [78]:
%%sql
-- to combine into a singular table
WITH category_scores AS (
    SELECT
        REPLACE(LOWER(c.category), '"', '') AS category,
        SUM(rt.Score) AS total_score
    FROM
        reactions r
        INNER JOIN content c ON r."Content ID" = c.content_id
        INNER JOIN reaction_type rt ON r.Type = rt.Type
    WHERE
        r."Content ID" IS NOT NULL
        AND r.Type IS NOT NULL
        AND r.Datetime IS NOT NULL
    GROUP BY
        category
    ORDER BY
        total_score
)

SELECT
    r."Content ID" AS content_id,
    LOWER(r.Type) AS reaction_type,
    r.Datetime AS datetime,
    c.content_type,
    REPLACE(LOWER(c.category), '"', '') AS category,
    rt.Sentiment AS sentiment,
    rt.Score AS score,
    cs.total_score
FROM
    reactions r
    INNER JOIN content c ON r."Content ID" = c.content_id
    INNER JOIN reaction_type rt ON r.Type = rt.Type
    LEFT JOIN category_scores cs ON REPLACE(LOWER(c.category), '"', '') = cs.category
WHERE
    r."Content ID" IS NOT NULL
    AND r.Type IS NOT NULL
    AND r.Datetime IS NOT NULL
ORDER BY cs.total_score DESC
LIMIT 10;


Running query in 'duckdb'

content_id,reaction_type,datetime,content_type,category,sentiment,score,total_score
07f88a73-aef2-45fd-8b5d-418e448b853d,cherish,2021-03-12 16:22:28,GIF,animals,positive,70,69548
07f88a73-aef2-45fd-8b5d-418e448b853d,intrigued,2020-10-17 10:20:24,GIF,animals,positive,45,69548
07f88a73-aef2-45fd-8b5d-418e448b853d,like,2020-12-19 16:42:35,GIF,animals,positive,50,69548
07f88a73-aef2-45fd-8b5d-418e448b853d,love,2021-05-31 05:46:59,GIF,animals,positive,65,69548
07f88a73-aef2-45fd-8b5d-418e448b853d,hate,2020-12-27 03:37:01,GIF,animals,negative,5,69548
07f88a73-aef2-45fd-8b5d-418e448b853d,scared,2021-05-24 20:32:26,GIF,animals,negative,15,69548
07f88a73-aef2-45fd-8b5d-418e448b853d,hate,2021-01-19 22:28:57,GIF,animals,negative,5,69548
07f88a73-aef2-45fd-8b5d-418e448b853d,cherish,2021-03-02 14:45:28,GIF,animals,positive,70,69548
07f88a73-aef2-45fd-8b5d-418e448b853d,adore,2021-02-12 16:37:09,GIF,animals,positive,72,69548
4fa14453-7b29-4302-b51f-9aa23b472c1b,cherish,2021-04-18 19:52:53,audio,animals,positive,70,69548


In [81]:
%%sql
-- export to csv
COPY (
WITH category_scores AS (
    SELECT
        REPLACE(LOWER(c.category), '"', '') AS category,
        SUM(rt.Score) AS total_score
    FROM
        reactions r
        INNER JOIN content c ON r."Content ID" = c.content_id
        INNER JOIN reaction_type rt ON r.Type = rt.Type
    WHERE
        r."Content ID" IS NOT NULL
        AND r.Type IS NOT NULL
        AND r.Datetime IS NOT NULL
    GROUP BY
        category
    ORDER BY
        total_score
)

SELECT
    r."Content ID" AS content_id,
    LOWER(r.Type) AS reaction_type,
    r.Datetime AS datetime,
    c.content_type,
    REPLACE(LOWER(c.category), '"', '') AS category,
    rt.Sentiment AS sentiment,
    rt.Score AS score,
    cs.total_score
FROM
    reactions r
    INNER JOIN content c ON r."Content ID" = c.content_id
    INNER JOIN reaction_type rt ON r.Type = rt.Type
    LEFT JOIN category_scores cs ON REPLACE(LOWER(c.category), '"', '') = cs.category
WHERE
    r."Content ID" IS NOT NULL
    AND r.Type IS NOT NULL
    AND r.Datetime IS NOT NULL
ORDER BY cs.total_score DESC
)
TO "../data/clean/final-clean.csv" (HEADER, DELIMITER ',');

Running query in 'duckdb'

Count
61869
